<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Data Scraping

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Preprocess the data
* Store the fight and fighter data into csv files

Functional as of April 2021

## Set parameters for dataset creation

NUM_EVENTS_INPUT: Integer number of UFC events to get fights from or "All" for all events. There are about 10 fights per event.

DATA_MODE_INPUT: Either "Summary" or "Round by Round". Either get data with columns that are summaries of the whole fight, or summaries round-by-round (more columns).

In [29]:
# NUM_EVENTS_INPUT = "All"  #@param {type:"string"}
NUM_EVENTS_INPUT = "2"  #@param {type:"string"}
DATA_MODE_INPUT = "Round by Round"  #@param {type:"string"} #changed from Summary

In [30]:
NUM_EVENTS = None if NUM_EVENTS_INPUT == "All" else int(NUM_EVENTS_INPUT)
ROUND_BY_ROUND = (DATA_MODE_INPUT == "Round by Round")

## Get information about all fighters

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
from string import ascii_lowercase
from bs4 import BeautifulSoup
import requests

In [4]:
def get_all_fighters():
    '''Get pandas table of all UFC fighters (Name, Height, Weight, Reach, Record, etc.)'''
    all_fighters_tables = []
    for c in tqdm(ascii_lowercase):
        all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
        all_fighters_table = pd.read_html(all_fighters_url)[0]
        all_fighters_tables.append(all_fighters_table)

    all_fighters = pd.concat(all_fighters_tables)
    return all_fighters

In [5]:
ALL_FIGHTERS = get_all_fighters()
ALL_FIGHTERS.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:45<00:00,  1.75s/it]


,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN


In [6]:
ALL_FIGHTERS.dtypes

First        object
Last         object
Nickname     object
Ht.          object
Wt.          object
Reach        object
Stance       object
W           float64
L           float64
D           float64
Belt        float64
dtype: object

## Clean fighter data

TODO: Convert height, weight, reach to floats.

In [7]:
ALL_FIGHTERS = ALL_FIGHTERS.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan
ALL_FIGHTERS.dropna(subset=["First", "Last"], how='all')  # Remove rows with no name
ALL_FIGHTERS.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,NaN,155 lbs.,NaN,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,NaN,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,NaN,Switch,10.0,15.0,0.0,NaN


## Helper functions

In [8]:
def get_fighters(fighters_string):
    '''Parses string containing two fighter names. Uses ALL_FIGHTERS global to remove ambiguity in parsing. Returns each fighter name
       Eg. "Robert Whittaker Kelvin Gastelum" => ("Robert Whittaker", "Kelvin Gastelum")'''
    for i, row in ALL_FIGHTERS.iterrows():
        fighter_name = f'{row["First"]} {row["Last"]}'
        if fighters_string.startswith(fighter_name):
            first_fighter = fighter_name
            second_fighter = fighters_string[len(fighter_name)+1:]
            break
    return first_fighter, second_fighter

def remove_duplicates_keep_order(list_):
    '''Removes duplicates while keeping same order'''
    return list(dict.fromkeys(list_))

## Get a list of all UFC events

In [9]:
from urllib.request import urlopen
from string import ascii_uppercase
from dateutil import parser
from datetime import datetime

In [10]:
ALL_PAST_EVENTS_URL = "http://ufcstats.com/statistics/events/completed?page=all"

In [11]:
def get_all_events(all_past_events_url):
    '''Takes in URL to all past events. Returns list of urls, each one representing a UFC event'''
    all_past_events_html = urlopen(all_past_events_url).read().decode("utf-8")
    
    # Regex for "http://ufcstats.com/events-details/<alphanumeric>"
    # Eg. "http://ufcstats.com/event-details/27541033b97c076d"
    pattern = "\"http://ufcstats.com/event-details/[a-zA-Z0-9_]+\""
    all_urls = re.findall(pattern, all_past_events_html)

    # Remove quotes and duplicates
    all_urls = [url.strip("\"") for url in all_urls]
    all_urls = remove_duplicates_keep_order(all_urls)
    return all_urls

In [12]:
# Events
ALL_EVENT_URLS = get_all_events(ALL_PAST_EVENTS_URL)
print(f"Got {len(ALL_EVENT_URLS)} events")
print()

print("Removing the most recent event, since it might not have happened yet")
ALL_EVENT_URLS = ALL_EVENT_URLS[1:]
print(f"Now got {len(ALL_EVENT_URLS)} events")
print(ALL_EVENT_URLS)

Got 651 events

Removing the most recent event, since it might not have happened yet
Now got 650 events
['http://ufcstats.com/event-details/b9415726dc3ec526', 'http://ufcstats.com/event-details/b6c6d1731ff00eeb', 'http://ufcstats.com/event-details/7abe471b61725980', 'http://ufcstats.com/event-details/6f812143641ceff8', 'http://ufcstats.com/event-details/901cddcbfa079097', 'http://ufcstats.com/event-details/3c6976f8182d9527', 'http://ufcstats.com/event-details/51b1e2fd9872005b', 'http://ufcstats.com/event-details/6fb1ba67bef41b37', 'http://ufcstats.com/event-details/15b1b21cd743d652', 'http://ufcstats.com/event-details/3dc3022232b79c7a', 'http://ufcstats.com/event-details/aec273fcb765330d', 'http://ufcstats.com/event-details/e4bb7e483c4ad318', 'http://ufcstats.com/event-details/35080a7f406f9ab3', 'http://ufcstats.com/event-details/1ccff7f0cfdf85eb', 'http://ufcstats.com/event-details/806975e1b4f47b27', 'http://ufcstats.com/event-details/f21a3d68fb9df387', 'http://ufcstats.com/event-deta

## Get a list of all UFC events from MMA Decisions

In [42]:
def get_all_mma_decision_events():
    '''Takes in URL to all past events, goes through each year and returns list if urls, each representing 
        a past event from MMA Decicions from newest to oldest'''
    all_href_event_links = []
    for year in range(2023, 1994, -1):
        all_yearly_events_url = f"http://mmadecisions.com/decisions-by-event/{year}/"
        response = requests.get(all_yearly_events_url)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all("a", href=lambda href: href and "UFC" in href)
        href_list = [link["href"] for link in links]
        all_href_event_links.extend(href_list)
    #all_href_event_links = list(chain.from_iterable(all_href_event_links))
    print(all_href_event_links)

In [43]:
get_all_mma_decision_events()

['event/1389/UFC-on-ESPN-47-Vettori-vs-Cannonier', 'event/1386/UFC-289-Nunes-vs-Aldana', 'event/1384/UFC-on-ESPN-46-Kara-France-vs-Albazi', 'event/1382/UFC-on-ESPN+-81-Dern-vs-Hill', 'event/1381/UFC-on-ABC-4-Rozenstruik-vs-Almeida', 'event/1379/UFC-288-Sterling-vs-Cejudo', 'event/1376/UFC-on-ESPN+-81-Song-vs-Simon', 'event/1373/UFC-on-ESPN+-80-Pavlovich-vs-Blaydes', 'event/1370/UFC-on-ESPN-44-Holloway-vs-Allen', 'event/1368/UFC-287-Pereira-vs-Adesanya-2', 'event/1364/UFC-on-ESPN-43-Vera-vs-Sandhagen', 'event/1362/UFC-286-Edwards-vs-Usman-3', 'event/1360/UFC-on-ESPN+-79-Yan-vs-Dvalishvili', 'event/1357/UFC-285-Jones-vs-Gane', 'event/1356/UFC-on-ESPN+-78-Muniz-vs-Allen', 'event/1354/UFC-on-ESPN+-77-Andrade-vs-Blanchfield', 'event/1353/UFC-284-Makhachev-vs-Volkanovski', 'event/1352/UFC-on-ESPN+-76-Lewis-vs-Spivak', 'event/1350/UFC-283-Teixeira-vs-Hill', 'event/1348/UFC-on-ESPN+-75-Strickland-vs-Imavov', 'event/1345/UFC-on-ESPN+-74-Cannonier-vs-Strickland', 'event/1343/UFC-282-Blachowicz-v

## Get a list of UFC fights from MMA Decisions

In [ ]:
def get_all_mma_decision_fights_in_event(past_event_url):
    '''Takes in a single URL to a past MMA Decision event.
        return fight_urls'''
    

In [ ]:
def get_all_fights_in_mma_decision(all_mma_event_urls, num_events=None):
    '''Takes in list of URLs to past events. Returns 3 lists: urls, winners, round by round scores, each representing a UFC fight.
    Set num_events to be the number of events to get fights from. Set to None if want all.'''
    if num_events is None:
        num_events = len(all_mma_event_urls)
    
    all_fight_urls, all_winners, all_rounds = [], [], []
    for i, event_url in enumerate(tqdm(all_mma_event_urls[:num_events])):
        # For each event, get the fight urls and winners
        event_url = "http://mmadecisions.com/" + event_url
        fight_urls, winners, rounds = get_all_fights_in_event(event_url, get_results=True)
        all_fight_urls.extend(fight_urls)
        all_winners.extend(winners)
        all_rounds.extend(methods)
    return all_fight_urls, all_winners, all_rounds

## Get a list of UFC fights

TODO: Right now only sees if result is win. Else sets winner to None. See if this can be improved.

In [13]:
def get_all_fights_in_event(past_event_url, get_results=False):
    '''Takes in a single URL to a past event.
       If get_results=True, returns fight_urls, winners, methods
       else, return fight_urls'''
    # Regex for "http://ufcstats.com/events-details/<alphanumeric>"
    # Eg. "http://ufcstats.com/fight-details/f67aa0b16e16a9ea"
    past_event_html = urlopen(past_event_url).read().decode("utf-8")
    pattern = "\"http://ufcstats.com/fight-details/[a-zA-Z0-9_]+\""
    fight_urls = re.findall(pattern, past_event_html)

    # Remove quotes and duplicates
    fight_urls = [url.strip("\"") for url in fight_urls]
    fight_urls = remove_duplicates_keep_order(fight_urls)

    # Get the winner and method (dec or KO or sub) of each fight
    past_event_table = pd.read_html(past_event_url)[0]  # Will be length 1 list
    winners, methods = [], []
    for _, row in past_event_table.iterrows():
        # TODO: Improve this processing of result
        result = row["W/L"].split(' ')[0]
        if result == "win":
            winner, _ = get_fighters(row["Fighter"])
        else:
            winner = None
        winners.append(winner)
        methods.append(row["Method"])

    if get_results:
        return fight_urls, winners, methods
    else:
        return fight_urls

In [14]:
def get_all_fights(all_event_urls, num_events=None):
    '''Takes in list of URLs to past events. Returns 3 lists: urls, winners, methods, each representing a UFC fight.
       Set num_events to be the number of events to get fights from. Set to None if want all.'''
    if num_events is None:
        num_events = len(all_event_urls)
    
    all_fight_urls, all_winners, all_methods = [], [], []
    for i, event_url in enumerate(tqdm(all_event_urls[:num_events])):
        # For each event, get the fight urls and winners
        fight_urls, winners, methods = get_all_fights_in_event(event_url, get_results=True)
        all_fight_urls.extend(fight_urls)
        all_winners.extend(winners)
        all_methods.extend(methods)
    return all_fight_urls, all_winners, all_methods

In [15]:
FIGHT_URLS, WINNERS, METHODS = get_all_fights(ALL_EVENT_URLS, num_events=NUM_EVENTS)
print(f"Got {len(FIGHT_URLS)} fights")
print(FIGHT_URLS)
print(WINNERS)
print(METHODS)

assert(len(FIGHT_URLS) == len(WINNERS))
assert(len(FIGHT_URLS) == len(METHODS))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.91s/it]

Got 23 fights
['http://ufcstats.com/fight-details/175d9e1ac0725145', 'http://ufcstats.com/fight-details/6c2199524d762b11', 'http://ufcstats.com/fight-details/5d9b84493f1ed114', 'http://ufcstats.com/fight-details/76fd6919d828d5c2', 'http://ufcstats.com/fight-details/98458778348ea5d7', 'http://ufcstats.com/fight-details/870672b1b443311e', 'http://ufcstats.com/fight-details/bc1be168c064f028', 'http://ufcstats.com/fight-details/7ba54a10f28c9496', 'http://ufcstats.com/fight-details/8ea12c7360fa0f6b', 'http://ufcstats.com/fight-details/9cfb84817a43b63c', 'http://ufcstats.com/fight-details/9a2d3ebab827d333', 'http://ufcstats.com/fight-details/8794031c523bf534', 'http://ufcstats.com/fight-details/69bc7ca8ce831731', 'http://ufcstats.com/fight-details/40e8bf8ce508c436', 'http://ufcstats.com/fight-details/e9d5ffcacf7d8757', 'http://ufcstats.com/fight-details/46c67efd68a0b15d', 'http://ufcstats.com/fight-details/5311298f79414125', 'http://ufcstats.com/fight-details/96eaf07b34538b8b', 'http://ufcst

## Get fight tables


In [16]:
def get_labeled_fight_tables(fight_url):
    '''Convert fight url to dictionary of pandas tables of information.
       Before, gave a list of tables that was hard to understand.
       Now have Totals, Per Round Totals, Significant Strikes, Per Round Significant Strikes'''
    fight_tables = pd.read_html(fight_url)
    
    labeled_fight_tables = {}
    labeled_fight_tables['Totals'] = fight_tables[0]
    labeled_fight_tables['Per Round Totals'] = fight_tables[1]
    labeled_fight_tables['Significant Strikes'] = fight_tables[2]
    labeled_fight_tables['Per Round Significant Strikes'] = fight_tables[3]
    return labeled_fight_tables

In [17]:
RAW_FIGHT_TABLES_LIST = []
for url in tqdm(FIGHT_URLS):
    RAW_FIGHT_TABLES_LIST.append(get_labeled_fight_tables(url))

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:17<00:00,  1.29it/s]


In [18]:
RAW_FIGHT_TABLES_LIST[0]['Totals'].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Marvin Vettori Jared Cannonier,0 0,153 of 301 241 of 411,50% 58%,154 of 302 257 of 428,1 of 1 4 of 6,100% 66%,0 0,0 0,0:28 3:07


In [19]:
RAW_FIGHT_TABLES_LIST[0]['Per Round Totals'].head()

Fighter      KD            Sig. str. Sig. str. %  \
                          Round 1 Round 1              Round 1     Round 1   
                          Round 2 Round 2              Round 2     Round 2   
                          Round 3 Round 3              Round 3     Round 3   
                          Round 4 Round 4              Round 4     Round 4   
                          Round 5 Round 5              Round 5     Round 5   
0  Marvin Vettori Jared Cannonier    0  0   38 of 71  29 of 66    53%  43%   
1  Marvin Vettori Jared Cannonier    0  0  24 of 50  82 of 125    48%  65%   
2  Marvin Vettori Jared Cannonier    0  0   31 of 50  49 of 81    62%  60%   
3  Marvin Vettori Jared Cannonier    0  0   37 of 76  32 of 57    48%  56%   
4  Marvin Vettori Jared Cannonier    0  0   23 of 54  49 of 82    42%  59%   

            Total str.            Td %            Sub. att    Rev.        Ctrl  
               Round 1         Round 1             Round 1 Round 1     Round 1  
               Round 2         Round 2             Round 2 Round 2     Round 2  
               Round 3         Round 3             Round 3 Round 3     Round 3  
               Round 4         Round 4             Round 4 Round 4     Round 4  
               Round 5         Round 5  Round 5.1  Round 5 Round 5     Round 5  
0   38 of 71  29 of 66  0 of 0  0 of 0   ---  ---     0  0    0  0  0:00  0:00  
1  24 of 50  94 of 138  0 of 0  0 of 0   ---  ---     0  0    0  0  0:00  0:46  
2   31 of 50  52 of 84  1 of 1  1 of 2  100%  50%     0  0    0  0  0:23  0:43  
3   37 of 76  32 of 57  0 of 0  1 of 2   ---  50%     0  0    0  0  0:00  0:34  
4   24 of 55  50 of 83  0 of 0  2 of 2  ---  100%     0  0    0  0  0:05  1:04

In [20]:
RAW_FIGHT_TABLES_LIST[0]['Significant Strikes'].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Marvin Vettori Jared Cannonier,153 of 301 241 of 411,50% 58%,117 of 257 173 of 339,15 of 21 39 of 43,21 of 23 29 of 29,148 of 293 195 of 354,5 of 8 26 of 31,0 of 0 20 of 26


In [21]:
RAW_FIGHT_TABLES_LIST[0]['Per Round Significant Strikes'].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3
,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4
,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5
0,Marvin Vettori Jared Cannonier,38 of 71 29 of 66,53% 43%,31 of 63 14 of 51,3 of 4 8 of 8,4 of 4 7 of 7,37 of 69 28 of 65,1 of 2 1 of 1,0 of 0 0 of 0,NaN
1,Marvin Vettori Jared Cannonier,24 of 50 82 of 125,48% 65%,21 of 44 62 of 104,2 of 4 15 of 16,1 of 2 5 of 5,22 of 47 58 of 96,2 of 3 12 of 13,0 of 0 12 of 16,NaN
2,Marvin Vettori Jared Cannonier,31 of 50 49 of 81,62% 60%,23 of 42 36 of 66,3 of 3 8 of 10,5 of 5 5 of 5,30 of 49 41 of 70,1 of 1 7 of 10,0 of 0 1 of 1,NaN
3,Marvin Vettori Jared Cannonier,37 of 76 32 of 57,48% 56%,26 of 63 22 of 46,4 of 5 3 of 4,7 of 8 7 of 7,37 of 75 32 of 57,0 of 1 0 of 0,0 of 0 0 of 0,NaN
4,Marvin Vettori Jared Cannonier,23 of 54 49 of 82,42% 59%,16 of 45 39 of 72,3 of 5 5 of 5,4 of 4 5 of 5,22 of 53 36 of 66,1 of 1 6 of 7,0 of 0 7 of 9,NaN


## Clean fight information

Separate each fighter's information into a different column

TODO: Lots of stuff to improve. Smarter use of Totals, round by round, and significant strikes. Can also use non integer information, total attempted strikes (not just landed), fighter information, etc. All of those being ignored right now. Find nice way to parse new information round by round. Handle no winner case better. May need to add ignore_index=True for pd.concat

In [22]:
def parse_string(row_string):
    '''Break string into two parts: one for fighter 0 and one for fighter 1
       Eg. 150 of 284  62 of 209 => (150 of 284, 62 of 209)'''
    if not isinstance(row_string, str):
        return "0", "0"
    string_split = row_string.split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [23]:
def convert_to_int_or_double_if_possible(string):
    '''Convert string to int or double if possible
       If has a percent sign, tries to remove it and continue.'''
    def isfloat(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    # If input is not string, then return it unchanged
    if not isinstance(string, str):
        return string

    # Remove %
    if "%" in string:
        string = string.strip("%")

    # Convert to int or float
    if isfloat(string) and float(string).is_integer():
        return int(string)
    if isfloat(string):
        return float(string)
    return string

In [38]:
def process_fight(raw_fight_table):
    '''Takes in a raw, one-row pandas fight table. Returns a pandas dataframe representing the fight statistics'''    
    # Break up columns.
    # Eg. "Name" => "Fighter 0 Name", "Fighter 1 Name"
    # "KD" => "Fighter 0 KD", "Fighter 1 KD"
    new_columns = []
    for column in raw_fight_table.columns:
        new_columns.append(f"Fighter 0 {column}")
        new_columns.append(f"Fighter 1 {column}")
    print("#########new column")
    print(new_columns)
    # Go through each row and break up the data into the columns
    new_rows = []
    for i, row in raw_fight_table.iterrows():
        new_row = []
        for column in raw_fight_table.columns:
            # Split string at the center space
            stat1, stat2 = parse_string(row[column])

            # TODO: Update this to capture more information

            # Has "100 of 120" type stat. Just store first number
            if " of " in stat1:
                stat1 = stat1.split(" of ")[0]
            if " of " in stat2:
                stat2 = stat2.split(" of ")[0]

            # Has "2:32" type stat (min:sec). Convert to sec.
            if len(re.findall("^[0-9]+:[0-9]+$", stat1)) > 0:
                min1, sec1 = stat1.split(":")[0], stat1.split(":")[1]
                stat1 = convert_to_int_or_double_if_possible(min1)*60 + convert_to_int_or_double_if_possible(sec1)
            if len(re.findall("^[0-9]+:[0-9]+$", stat2)) > 0:
                min2, sec2 = stat2.split(":")[0], stat2.split(":")[1]
                stat2 = convert_to_int_or_double_if_possible(min2)*60 + convert_to_int_or_double_if_possible(sec2)
            
            # Convert string to float or int if possible
            stat1 = convert_to_int_or_double_if_possible(stat1)
            stat2 = convert_to_int_or_double_if_possible(stat2)

            # Add to row
            new_row.append(stat1)
            new_row.append(stat2)
        new_rows.append(new_row)

    # Bring together into new dataframe, then only store the numerical values
    # TODO: Process better to keep more info, not throw so much away
    df = pd.DataFrame(new_rows, columns=new_columns)

    # Add in names, using smarter parsing
    df = df.drop(columns=['Fighter 0 Fighter', 'Fighter 1 Fighter'])
    fighters_string = raw_fight_table["Fighter"][0]  # Only 1 row table
    fighter0, fighter1 = get_fighters(fighters_string)
    df['Fighter 0 Name'] = fighter0
    df['Fighter 1 Name'] = fighter1
    return df

In [40]:
def process_raw_fight_tables(raw_fight_tables, winner, method, round_by_round=False):
    '''Takes in set of raw fight table (one fight), the name of the fight winner, and the method of winning. Returns a cleaned pandas table.
       Set round_by_round=True to use the round-by-round data. Otherwise, uses full fight stats.'''
    def create_aggregated_fight_table(raw_fight_tables):
        # Aggregate data from multiple tables
        fight_table = process_fight(raw_fight_tables["Totals"])
        fight_table2 = process_fight(raw_fight_tables["Significant Strikes"])
        
        # Rename column names with identical data to match
        fight_table2 = fight_table2.rename(columns={"Fighter 0 Sig. str": "Fighter 0 Sig. str.", "Fighter 1 Sig. str": "Fighter 1 Sig. str."})

        # Bring tables together, then remove duplicates
        fight_table = pd.concat([fight_table, fight_table2], axis=1)
        fight_table = fight_table.loc[:,~fight_table.columns.duplicated()]
        return fight_table

    def create_aggregated_round_by_round_fight_table(raw_fight_tables):
        ##### Aggregate data totals table
        tables = []
        print(raw_fight_tables)
        for i, row in raw_fight_tables["Per Round Totals"].iterrows():
            # Get df of one round
            df = pd.DataFrame(row)
            print("######################")
            print("before")
            print(df)
            print("after")
            values = list(df[i].to_dict().values())
            cols = list(raw_fight_tables["Totals"].columns)
            print("cols##########")
            print(cols)
            df = pd.DataFrame([values], columns=cols)
            print(df)
            # Update columns with round number
            new_cols = [f"Round {i+1} {c}" if  bnc != "Fighter" else c for c in cols]
            df.columns = new_cols
            print("#####updated")
            print(df)
            tables.append(process_fight(df))
        # Concatenate round-by-round horizontally, so each row is for 1 fight.
        # Then remove duplicates
        totals_df = pd.concat(tables, axis=1)
        totals_df = totals_df.loc[:,~totals_df.columns.duplicated()]
        print("########final totals_df")
        print(totals_df)

        ##### Aggregate data significant strikes table
        tables = []
        for i, row in raw_fight_tables["Per Round Significant Strikes"].iterrows():
            # Get df of one round
            df = pd.DataFrame(row)
            values = list(df[i].to_dict().values())
            cols = list(raw_fight_tables["Significant Strikes"].columns)
            if len(values) != len(cols):
                values = values[:-1]  # Remove last column values, as shown above, has extra column for no reason
            df = pd.DataFrame([values], columns=cols)

            # Update columns with round number
            new_cols = [f"Round {i+1} {c}" if c != "Fighter" else c for c in cols]
            df.columns = new_cols
            tables.append(process_fight(df))
        # Concatenate round-by-round horizontally, so each row is for 1 fight
        # Then remove duplicates
        sig_strikes_df = pd.concat(tables, axis=1)
        sig_strikes_df = sig_strikes_df.loc[:,~sig_strikes_df.columns.duplicated()]
        
        ##### Bring tables together, then remove duplicates
        fight_table = pd.concat([totals_df, sig_strikes_df], axis=1)
        fight_table = fight_table.loc[:,~fight_table.columns.duplicated()]
        return fight_table


    if round_by_round:
        fight_table = create_aggregated_round_by_round_fight_table(raw_fight_tables)
    else:
        fight_table = create_aggregated_fight_table(raw_fight_tables)

    if fight_table["Fighter 0 Name"][0] == winner:
        label = 0
    elif fight_table["Fighter 1 Name"][0] == winner:
        label = 1
    else:
        print(f'ERROR: fight_table["Fighter 0 Name"]={fight_table["Fighter 0 Name"]}, fight_table["Fighter 1 Name"]={fight_table["Fighter 1 Name"]}, winner={winner}')
        label = -1
    fight_table['Winner'] = label
    fight_table['Method'] = method
    return fight_table

In [41]:
FIGHT_TABLE = []
for i in tqdm(range(len(RAW_FIGHT_TABLES_LIST))):
    FIGHT_TABLE.append(process_raw_fight_tables(RAW_FIGHT_TABLES_LIST[i], WINNERS[i], METHODS[i], round_by_round=ROUND_BY_ROUND)) 
FIGHT_TABLE = pd.concat(FIGHT_TABLE, ignore_index=True)
FIGHT_TABLE = FIGHT_TABLE.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan

  0%|                                                                                           | 0/23 [00:00<?, ?it/s]

{'Totals':                           Fighter    KD               Sig. str. Sig. str. %  \
0  Marvin Vettori Jared Cannonier  0  0  153 of 301  241 of 411    50%  58%   

               Total str.              Td       Td % Sub. att  Rev.  \
0  154 of 302  257 of 428  1 of 1  4 of 6  100%  66%     0  0  0  0   

         Ctrl  
0  0:28  3:07  , 'Per Round Totals':                           Fighter      KD            Sig. str. Sig. str. %  \
                          Round 1 Round 1              Round 1     Round 1   
                          Round 2 Round 2              Round 2     Round 2   
                          Round 3 Round 3              Round 3     Round 3   
                          Round 4 Round 4              Round 4     Round 4   
                          Round 5 Round 5              Round 5     Round 5   
0  Marvin Vettori Jared Cannonier    0  0   38 of 71  29 of 66    53%  43%   
1  Marvin Vettori Jared Cannonier    0  0  24 of 50  82 of 125    48%  65%   
2  Marvin 

######################
before
                                                                                    2
Fighter     Round 1 Round 2 Round 3 Round 4 Round 5    Marvin Vettori Jared Cannonier
KD          Round 1 Round 2 Round 3 Round 4 Round 5                              0  0
Sig. str.   Round 1 Round 2 Round 3 Round 4 Round 5                31 of 50  49 of 81
Sig. str. % Round 1 Round 2 Round 3 Round 4 Round 5                          62%  60%
Total str.  Round 1 Round 2 Round 3 Round 4 Round 5                31 of 50  52 of 84
Td %        Round 1 Round 2 Round 3 Round 4 Round 5                    1 of 1  1 of 2
                                            Round 5.1                       100%  50%
Sub. att    Round 1 Round 2 Round 3 Round 4 Round 5                              0  0
Rev.        Round 1 Round 2 Round 3 Round 4 Round 5                              0  0
Ctrl        Round 1 Round 2 Round 3 Round 4 Round 5                        0:23  0:43
after
cols##########
['F

#########new column
['Fighter 0 Fighter', 'Fighter 1 Fighter', 'Fighter 0 Round 2 Sig. str', 'Fighter 1 Round 2 Sig. str', 'Fighter 0 Round 2 Sig. str. %', 'Fighter 1 Round 2 Sig. str. %', 'Fighter 0 Round 2 Head', 'Fighter 1 Round 2 Head', 'Fighter 0 Round 2 Body', 'Fighter 1 Round 2 Body', 'Fighter 0 Round 2 Leg', 'Fighter 1 Round 2 Leg', 'Fighter 0 Round 2 Distance', 'Fighter 1 Round 2 Distance', 'Fighter 0 Round 2 Clinch', 'Fighter 1 Round 2 Clinch', 'Fighter 0 Round 2 Ground', 'Fighter 1 Round 2 Ground']
#########new column
['Fighter 0 Fighter', 'Fighter 1 Fighter', 'Fighter 0 Round 3 Sig. str', 'Fighter 1 Round 3 Sig. str', 'Fighter 0 Round 3 Sig. str. %', 'Fighter 1 Round 3 Sig. str. %', 'Fighter 0 Round 3 Head', 'Fighter 1 Round 3 Head', 'Fighter 0 Round 3 Body', 'Fighter 1 Round 3 Body', 'Fighter 0 Round 3 Leg', 'Fighter 1 Round 3 Leg', 'Fighter 0 Round 3 Distance', 'Fighter 1 Round 3 Distance', 'Fighter 0 Round 3 Clinch', 'Fighter 1 Round 3 Clinch', 'Fighter 0 Round 3 Ground'

  4%|███▌                                                                               | 1/23 [00:02<01:04,  2.91s/it]

...

 91%|██████████████████████████████████████████████████████████████████████████▊       | 21/23 [00:17<00:01,  1.32it/s]

{'Totals':                     Fighter    KD             Sig. str. Sig. str. %  \
0  David Dvorak Steve Erceg  0  0  53 of 114  54 of 140    46%  38%   

             Total str.              Td      Td % Sub. att  Rev.        Ctrl  
0  56 of 117  56 of 142  1 of 3  3 of 4  33%  75%     0  1  0  0  1:57  1:29  , 'Per Round Totals':                     Fighter      KD           Sig. str. Sig. str. %  \
                    Round 1 Round 1             Round 1     Round 1   
                    Round 2 Round 2             Round 2     Round 2   
                    Round 3 Round 3             Round 3     Round 3   
0  David Dvorak Steve Erceg    0  0  20 of 46  16 of 56    43%  28%   
1  David Dvorak Steve Erceg    0  0  23 of 49  18 of 51    46%  35%   
2  David Dvorak Steve Erceg    0  0  10 of 19  20 of 33    52%  60%   

           Total str.            Td %            Sub. att    Rev.        Ctrl  
              Round 1         Round 1             Round 1 Round 1     Round 1  
         

########final totals_df
   Fighter 0 Round 1 KD  Fighter 1 Round 1 KD  Fighter 0 Round 1 Sig. str.  \
0                     0                     0                           20   

   Fighter 1 Round 1 Sig. str.  Fighter 0 Round 1 Sig. str. %  \
0                           16                             43   

   Fighter 1 Round 1 Sig. str. %  Fighter 0 Round 1 Total str.  \
0                             28                            20   

   Fighter 1 Round 1 Total str.  Fighter 0 Round 1 Td  Fighter 1 Round 1 Td  \
0                            16                     0                     0   

   ... Fighter 0 Round 3 Td Fighter 1 Round 3 Td  Fighter 0 Round 3 Td %  \
0  ...                    1                    3                      33   

   Fighter 1 Round 3 Td %  Fighter 0 Round 3 Sub. att  \
0                     100                           0   

   Fighter 1 Round 3 Sub. att  Fighter 0 Round 3 Rev.  Fighter 1 Round 3 Rev.  \
0                           0                  

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:18<00:00,  1.27it/s]

{'Totals':                         Fighter    KD              Sig. str. Sig. str. %  \
0  Diana Belbita Maria Oliveira  0  0  106 of 208  64 of 168    50%  38%   

              Total str.              Td      Td % Sub. att  Rev.        Ctrl  
0  154 of 258  85 of 200  1 of 2  2 of 5  50%  40%     0  0  0  0  2:04  4:06  , 'Per Round Totals':                         Fighter      KD           Sig. str. Sig. str. %  \
                        Round 1 Round 1             Round 1     Round 1   
                        Round 2 Round 2             Round 2     Round 2   
                        Round 3 Round 3             Round 3     Round 3   
0  Diana Belbita Maria Oliveira    0  0  41 of 84  28 of 66    48%  42%   
1  Diana Belbita Maria Oliveira    0  0   22 of 37  7 of 17    59%  41%   
2  Diana Belbita Maria Oliveira    0  0  43 of 87  29 of 85    49%  34%   

            Total str.            Td %            Sub. att    Rev.        Ctrl  
               Round 1         Round 1          

In [ ]:
FIGHT_TABLE.head()

In [ ]:
FIGHT_TABLE.tail()

## Augment dataset by flipping around columns

The system should work the same no matter what order we pass in the fighters. Let fighters be A and B. We want

winner(fighter0=A, fighter1=B) = winner(fighter0=B, fighter1=A)

In [ ]:
def create_flipped_table(table):
    '''Rearranges columns of table so that each fight has two rows. Let fighters be A and B.
       One row has (Fighter 0 = A, Fighter 1 = B). One row has (Fighter 0 = B, Fighter 1 = A)
       Ensure same column order, as column names not looked at when passed to ML model'''

    # Get columns in flipped order, which moves the columns around, but changes column name order too
    flipped_columns = []
    for column in table.columns:
        if "Fighter 0" in column:
            flipped_columns.append(column.replace("Fighter 0", "Fighter 1"))
        elif "Fighter 1" in column:
            flipped_columns.append(column.replace("Fighter 1", "Fighter 0"))
        else:
            flipped_columns.append(column)
    flipped_table = table[flipped_columns]

    # Flips winners around
    if 'Winner' in flipped_table.columns:
         flipped_table['Winner'] = flipped_table['Winner'].replace([0, 1], [1, 0])

    # Change column names back to normal
    flipped_table.columns = table.columns
    return flipped_table

In [ ]:
def add_rows_of_flipped_columns(table):
    flipped_table = create_flipped_table(table)
    new_table = pd.concat([table, flipped_table])
    return new_table

In [ ]:
FULL_FIGHT_TABLE = add_rows_of_flipped_columns(FIGHT_TABLE)

In [ ]:
FULL_FIGHT_TABLE.head()

## Example of augmented data

In [ ]:
FULL_FIGHT_TABLE[(FULL_FIGHT_TABLE['Fighter 0 Name'] == "Robert Whittaker") & (FULL_FIGHT_TABLE['Fighter 1 Name'] == "Kelvin Gastelum")]

In [ ]:
FULL_FIGHT_TABLE[(FULL_FIGHT_TABLE['Fighter 1 Name'] == "Robert Whittaker") & (FULL_FIGHT_TABLE['Fighter 0 Name'] == "Kelvin Gastelum")]

## Additional data cleaning

TODO: See if something better than replacing nan with 0. See if something better for labels than 0 and 1. Could remove fights with no winner, or handle them differently. Could remove fights that don't go to decision by removing based on Method.

In [ ]:
X = FIGHT_TABLE.drop(['Winner', 'Fighter 0 Name', 'Fighter 1 Name', 'Method'], axis=1).fillna(0)
y = FIGHT_TABLE[['Winner']]

In [ ]:
X.head()

In [ ]:
y.head()

## Setup train/validate/test split
Can't blindly use full fight table train/validate/test split, because the augmented data must stay together. If in train we know winner(A, B) = A, then we don't want to have winner(B, A) in the validation/test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=0)
X_train, y_train = add_rows_of_flipped_columns(X_train), add_rows_of_flipped_columns(y_train)
X_valid, y_valid = add_rows_of_flipped_columns(X_valid), add_rows_of_flipped_columns(y_valid)
X_test, y_test = add_rows_of_flipped_columns(X_test), add_rows_of_flipped_columns(y_test)

In [ ]:
# Expect equal number of examples in Fighter 0 as Fighter 1
assert(len(y_train[y_train['Winner'] == 0]) == len(y_train[y_train['Winner'] == 1]))
assert(len(y_valid[y_valid['Winner'] == 0]) == len(y_valid[y_valid['Winner'] == 1]))
assert(len(y_test[y_test['Winner'] == 0]) == len(y_test[y_test['Winner'] == 1]))

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_valid.shape = {X_valid.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"y_valid.shape = {y_valid.shape}")
print(f"y_test.shape = {y_test.shape}")

## ML Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Train
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)

# Validate
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
import matplotlib.pyplot as plt

# Visualize importances
plt.rcParams.update({'font.size': 8})
plt.barh(X_train.columns, clf.feature_importances_)

In [ ]:
# MLP
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
# SVM
from sklearn.svm import SVC

clf = SVC(random_state=1).fit(X_train, y_train)
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
# FFN
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=X_train.shape[1:]))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_train, y_train)
model.evaluate(X_valid, y_valid)

## Test out model manually

In [ ]:
idx = 6

In [ ]:
X_test.iloc[idx]

In [ ]:
# 0 means fighter 0 won. 1 means fighter 1 won.
y_test.iloc[idx]

In [ ]:
X_test.shape

In [ ]:
X_test.iloc[idx].shape

In [ ]:
model.predict(np.expand_dims(X_test.iloc[idx], 0))

## Save data

Store beginning file parameters.
Use current date and time to save files uniquely.

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")
print("dt_string =", dt_string)	

In [ ]:
parameters_string = f"NUM_EVENTS_{NUM_EVENTS_INPUT}_DATA_MODE_{DATA_MODE_INPUT}"
print("parameters_string =", parameters_string)	

In [ ]:
import pickle
filename1 = f"FULL_FIGHT_TABLE_{parameters_string}_{dt_string}.csv"
filename2 = f"FIGHT_TABLE_{parameters_string}_{dt_string}.csv"
filename3 = f"ALL_FIGHTERS_{parameters_string}_{dt_string}.csv"
filename4 = f"RAW_FIGHT_TABLES_LIST_{parameters_string}_{dt_string}.pkl"
print(f"Saving to {filename1} and {filename2} and {filename3} and {filename4}")
FULL_FIGHT_TABLE.to_csv(filename1, index=False)
FIGHT_TABLE.to_csv(filename2, index=False)
ALL_FIGHTERS.to_csv(filename3, index=False)
with open(filename4, 'wb') as handle:
    pickle.dump(RAW_FIGHT_TABLES_LIST, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
new = pd.read_csv(filename1)

In [ ]:
new

In [ ]:
with open(filename4, 'rb') as pickle_file:
    new2 = pickle.load(pickle_file)

In [ ]:
len(new2[0])

## Experimental: Get detailed fighter information

TODO: Get more detailed information about fighters, so we can change the task to fight prediction using fighter stats only. http://ufcstats.com/statistics/fighters?char=a&page=all has little information compared to http://ufcstats.com/fighter-details/33a331684283900f. Still lots to improve. Better features like strikes per minute. Handling nans better. Handling non win/losses better.

In [ ]:
def get_all_fighters_detailed():
    '''Get pandas table with detailed information about all UFC fighters (KO's, strikes, etc.)'''
    fighter_detailed_tables = []
    
    # For each letter of the alphabet, get the fighters
    for c in tqdm(ascii_lowercase):
        # Each page has a list of fighter detail urls
        all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
        all_fighters_html = urlopen(all_fighters_url).read().decode("utf-8")

        # Regex for "http://ufcstats.com/fighter-details/<alphanumeric>"
        # Eg. "http://ufcstats.com/fighter-details/27541033b97c076d"
        pattern = "\"http://ufcstats.com/fighter-details/[a-zA-Z0-9_]+\""
        urls = re.findall(pattern, all_fighters_html)
        
        # Remove quotes and duplicates
        urls = [url.strip("\"") for url in urls]
        urls = remove_duplicates_keep_order(urls)
        
        # For each fighter detail url, merge together their record information
        # Initially in form "Eddie Alvarez Rafael Dos Anjos", "0 0", "1:10, 0:00"
        # Want just "Eddie Alvarez", "0", "1:10", then convert to numbers
        # Just need to get the first value of each one, then average/sum/aggregate this together
        for url in urls:
            fighter_table = pd.read_html(url)[0].dropna(subset=["Time"], how='all')  # Drop initial row of nans

            # If no fight information, add empty dataframe
            if fighter_table.shape[0] == 0:
                df = pd.DataFrame()
                fighter_detailed_tables.append(df)
                continue
                
            # Preprocess certain values for consistency
            # TODO: Handle this better, perhaps keep more information
            fighter_table = fighter_table.drop(columns=["Method", "Event"])
            fighter_table.loc[~fighter_table['W/L'].isin(['win', 'loss']), 'W/L'] = "-1 -1"
            fighter_table.loc[fighter_table['W/L'] == 'win', 'W/L'] = "1  1"
            fighter_table.loc[fighter_table['W/L'] == 'loss', 'W/L'] = "0  0"
            times = [int(min_) * 60 + int(sec) for min_, sec in fighter_table['Time'].str.split(':')]
            fighter_table['Time'] = [f"{t}  {t}" for t in times]
            
            # Parse each row to remove the other fighter's information
            new_rows = []
            for i, row in fighter_table.iterrows():
                # Get df of one round
                df = pd.DataFrame(row, columns=fighter_table.columns)
                values = [row[col] for col in df.columns]
                df = pd.DataFrame([values], columns=fighter_table.columns)
                df = process_fight(df)
                new_rows.append(df)

            # Put rows together, then only keep Fighter 0, then remove "Fighter 0 "
            totals_df = pd.concat(new_rows)
            totals_df = totals_df.loc[:, totals_df.columns.str.contains('Fighter 0')]
            totals_df.columns = [col.replace("Fighter 0 ", "") for col in totals_df.columns]
            totals_df = totals_df.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan

            # Summarize fighter in 1 row
            new_columns = []
            new_row = []
            for col in totals_df.columns:
                if col == "Name":
                    new_columns.append(col)
                    new_row.append(totals_df[col].iloc[0])
                else:
                    total_col = f"{col} Total"
                    avg_col = f"{col} Avg"
                    new_columns.extend([total_col, avg_col])
                    total = totals_df[col].sum()
                    avg = totals_df[col].mean()
                    new_row.extend([total, avg])
            totals_df = pd.DataFrame([new_row], columns=new_columns)

            fighter_detailed_tables.append(totals_df) 
            break  # Remove this when ready
    all_fighters = pd.concat(fighter_detailed_tables)
    return all_fighters

In [ ]:
x = get_all_fighters_detailed()

In [ ]:
x.head()